### 11.1图1-实现

In [ ]:
# 顶点Vertex类
class Vertex:
    def __init__(self,key):
        self.id=key
        self.connectedTo={}
        # 添加BFS属性
        self.color='white'
        self.distance=0
        self.predecessor=None
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr]=weight
    def __str__(self):
        return str(self.id)+'connectedTo:'\
               + str([x.id for x in self.connectedTo])
    def getConnections(self):
        return self.connectedTo.keys()
    def getId(self):
        return self.id
    def getWeight(self,nbr):
        return self.connectedTo[nbr]
    # 添加BFS放法
    def setColor(self,color):
        self.color=color
    def getColor(self):
        return self.color
    def sefDistance(self,distance):
        self.distance=distance
    def getDistance(self):
        return self.distance
    def setPred(self,predecessor):
        self.predecessor=predecessor
    def getPred(self):
        return self.predecessor

# 图Graph类
class Graph:
    def __init__(self):
        self.vertList={}
        self.numVertices=0
    def addVertex(self,key):
        self.numVertices+=1
        newVertex=Vertex(key)
        self.vertList[key]=newVertex
        return newVertex
    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None
    def __contains__(self,n):
        return n in self.vertList
    def addEdge(self,f,t,cost=0):
        if f not in self.vertList:
            nv=self.addVertex(f)
        if t not in self.vertList:
            nv=self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t],cost)
    def getVertices(self):
        return self.vertList.keys()
    def __iter__(self):
        return iter(self.vertList.values())

### 11.4图的应用1-词梯问题

In [ ]:
def buildGraph(wordFile):
    d={}
    g=Graph()
    wfile=open(wordFile,'r')
    #create buckests of words that differ be one letter
    for line in wfile:
        word=line[:-1]
        for i in range(len(word)):
            bucket=word[:i]+'_'+word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket]=[word]
    #add vertices and edges for words in the same bucket
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1!=word2:
                    g.addEdge(word1,word2)
    return g

#### 广度优先搜索

In [ ]:
def bfs(g,start):
    start.setDistance(0)
    start.setPred(None)
    vertQueue=Queue()
    vertQueue.enqueue(start)
    while(vertQueue.size()>0):
        currentVert=vertQueue.dequeue()
        for nbr in currentVert.getConnections():
            if (nbr.getColor()=='white'):
                nbr.setColor('gray')
                nbr.setDistance(currentVert.getDistance())
                nbr.setPred(currentVert)
                vertQueue.enqueue(nbr)
        currentVert.setColor('black')
def traverse(y):
    x=y
    while(x.getPred()):
        print(x.getId())
        x=x.getPred()
    print(x.getId)

In [ ]:
#补充queue
class Queue:
    def __init__(self):
        self.items=[]
    def isEmpty(self):
        return self.items==[]
    def enqueue(self,item):
        self.items.insert(0,item)
    def dequeue(self):
        return self.items.pop()
    def size(self):
        return len(self.items)

#### 词梯最终搜索

In [ ]:
wordgraph=buildGraph("wordFile.txt")
bfs(wordgraph,wordgraph.getVertex('FOOL'))
traverse(wordgraph.getVertex('SAGE'))

### 11.6图的应用2-骑士周游问题

In [ ]:
def genLegalMoves(x,y,bdSize):
    newMoves=[]
    # 马走日的8个方向
    moveOffsets=[(-1,-2),(-1,2),(-2,-1),(-2,1),
                 (1,-2),(1,2),(2,-1),(2,1)]
    for i in moveOffsets:
        newX=x+i[0]
        newY=y+i[1]
        if legalCoord(newX,bdSize) and legalCoord(newY,bdSize):
            newMoves.append((newX,newY))
    return newMoves
#确认不会走出棋盘
def legalCoord(x,bdSize):
    if x>=0 and x<bdSize:
        return True
    else:
        return False
def knightGraph(bdSize):
    ktGraph=Graph()
    for row in range(bdSize):
        #遍历每一个格子
        for col in range(bdSize):
            nodeId=posToNodeId(row,col,bdSize)
            #单步合法走棋
            newPositions=genLegalMoves(row,col,bdSize)
            for e in newPositions:
                nid=posToNodeId(e[0],e[1],bdSize)
                #添加边及顶点
                ktGraph.addEdge(nodeId,nid)
    return ktGraph
def posToNodeId(row,col,bdSize):
    return row*bdSize+col

#### 深度优先算法DFS解决骑士周游问题

In [ ]:
def knightTour(n,path,u,limit): #n:层次；path:路径；u:当前顶点；limit:搜索总深度
    u.setColor('gray')
    #当前顶点加入路径
    path.append(u)
    if n<limit:
        #对所有合法移动逐一深入
        nbrList=list(u.getConnections())
        i=0
        done=False
        while i<len(nbrList) and not done:
            #选择白色未经过的顶点深入
            if nbrList[i].getColor()=='white':
                done=knightTour(n+1,path,nbrList[i],limit)
                #层次加1，递归深入
            i+=1
        if not done: #prepare to backtrack
            path.pop()
            #都无法完成总深度，回溯，尝试本层下一个顶点
            u.setColor('white')
    else:
        done=True
    return done

#### 深度优先算法的优化：Warnsdorff算法

In [ ]:
def orderByAvail(n):
    resList=[]
    for v in n.getConnections():
        if v.getColor()=='white':
            c=0
            for w in v.getConnections():
                if w.getColor=='white':
                    c+=1
            resList.append((c,v))
    resList.sort(key=lambda x:x[0])
    return [y[1] for y in resList]